In [64]:
!pip install duckdb

In [65]:
import duckdb

In [66]:
con = duckdb.connect(database= ':memory:', read_only=False)

In [67]:
query = "SELECT * FROM 'customers.csv'"

In [68]:
result = con.execute(query).fetchdf()

In [69]:
result.head()

,customer_id,first_name,last_name,email,created_at,created_date,status,country
0,1001,John,Doe,john.doe@example.com,2024-01-15 08:23:45,2024-01-15,active,US
1,1002,Emily,Smith,emily.smith@example.co.uk,2024-02-10 14:12:30,2024-02-10,active,UK
2,1003,Michael,Brown,michael.brown@example.com,2024-03-05 19:45:12,2024-03-05,active,US
3,1004,Sophia,Wilson,sophia.wilson@example.co.th,2024-04-01 09:05:55,2024-04-01,active,TH
4,1005,David,Johnson,david.johnson@example.com,2024-06-20 11:30:00,2024-06-20,active,US


In [70]:
result.shape

(10, 8)

In [71]:
filname = "Titanic-Dataset.csv"

In [72]:
query = f"SELECT * FROM '{filname}'"

In [73]:
Titanic_result = con.execute(query).fetchdf()

In [74]:
Titanic_result.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [75]:
Titanic_result.shape

(891, 12)

In [76]:
Titanic_result.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [77]:
query_json = "SELECT * FROM read_json_auto('products.json')"

In [78]:
product_result = con.execute(query_json).fetchdf()

In [79]:
query_multi = """
  select customer_id, first_name, last_name, email, created_date
  from 'customers.csv'
  where status = 'active' and created_date >= '2024-01-01' and country = 'TH'


"""

In [80]:
customer_result = con.execute(query_multi).fetchdf()

In [81]:
product_result = con.execute(query_multi).fetchdf()

In [82]:
query_titanic = f"""
  select PassengerID, Name, Age, Fare, case
    when Survived = 1 then 'survived'
    else 'not_survived'
  end as survived_status
  from 'Titanic-Dataset.csv'

"""

In [83]:
trandfrorm_result = con.execute(query_titanic).fetchdf()

In [84]:
trandfrorm_result.head()

,PassengerId,Name,Age,Fare,survived_status
0,1,"Braund, Mr. Owen Harris",22.0,7.2500,not_survived
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,71.2833,survived
2,3,"Heikkinen, Miss. Laina",26.0,7.9250,survived
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,53.1000,survived
4,5,"Allen, Mr. William Henry",35.0,8.0500,not_survived


In [85]:
query = "select *from 'orders.csv'"

order_result = con.execute(query).fetchdf()


In [86]:
order_result.columns

Index(['order_id', 'customer_id', 'amount', 'order_date', 'status'], dtype='object')

In [87]:
query = """
  select
  c.customer_id,
  c.customer_name,
  c.segment,
  count(o.order_id) as total_order,
  sum(o.sales) as total_sales

  from 'new_customers.csv' c
  join 'new_orders.csv' o
    on c.customer_id = o.customer_id
    and o.status != 'cancelled'
    where c.is_active = true
    group by 1,2,3


"""

In [88]:
query = "select *from 'new_customers.csv'"
new_customers_result = con.execute(query).fetchdf()

In [89]:
customers_metric = con.execute(query).fetchdf()

In [90]:
customers_metric.head()

,customer_id,customer_name,segment,is_active
0,C001,John Doe,Consumer,True
1,C002,Jane Smith,Corporate,True
2,C003,Acme Corp,SMB,True
3,C004,Inactive Inc,Consumer,False
4,C005,Global LLC,Enterprise,True


In [91]:
query = """
WITH
active_orders AS (
    SELECT
        customer_id,
        count(*) as order_count,
        sum(amount) as total_amount,
        avg(amount) as avg_amount
    FROM 'orders.csv'
    WHERE order_date >= date_add(CURRENT_DATE, INTERVAL '-12 month')
    GROUP BY 1
),
active_customers AS (
    SELECT
      customer_id,
      customer_name,
      segment
      FROM 'new_customers.csv'
    WHERE is_active = true
),
final_result AS (
    SELECT
      ac.*,
      COALESCE(ao.order_count, 0) as orders_count,
      COALESCE(ao.total_amount, 0) as total_amount,
      COALESCE(ao.avg_amount, 0) as avg_amount
       from active_customers ac
       left join active_orders ao
         on ac.customer_id = ao.customer_id
)

SElECT * FROM final_result;
"""

In [92]:
result2 = con.execute(query).fetchdf()

In [93]:
result2.head()

,customer_id,customer_name,segment,orders_count,total_amount,avg_amount
0,C007,Cancelled Cust,Consumer,1,100.0,100.0
1,C009,Future Buyer,Enterprise,1,1000.0,1000.0
2,C010,Small Biz,SMB,1,45.0,45.0
3,C001,John Doe,Consumer,0,0.0,0.0
4,C002,Jane Smith,Corporate,0,0.0,0.0


In [94]:
query = """
WITH enriched_orders AS (
    SELECT
        o.*,
        c.customer_name,
        c.segment
    FROM orders.csv o
    LEFT JOIN new_customers.csv c
        ON o.customer_id = c.customer_id
)
SELECT
    order_id,
    customer_id,
    customer_name,
    order_date,
    amount,

    -- 1. ลำดับของ order ของลูกค้าแต่ละคน
    ROW_NUMBER() OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as order_sequence,

    -- 2. อันดับยอดขายในเดือนนั้น
    RANK() OVER (
        PARTITION BY strftime('%Y-%m', order_date)
        ORDER BY amount DESC
    ) as monthly_sales_rank,

    -- 3. ยอดสะสมของลูกค้า
    SUM(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as running_total,

    -- 4. ค่าเฉลี่ยเคลื่อนที่ 4 orders
    AVG(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) as moving_avg_4_orders,

    -- 5. ยอด order ก่อนหน้า
    LAG(amount, 1, 0) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as previous_order,

    -- 6. ยอด order ถัดไป
    LEAD(amount, 1, 0) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as next_order,

    -- 7. ยอด order แรกของลูกค้า
    FIRST_VALUE(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as first_order_amount

FROM enriched_orders
ORDER BY customer_id, order_date;
"""


In [95]:
window_analysis = con.execute(query).fetchdf()

In [96]:
window_analysis.head()

,order_id,customer_id,customer_name,order_date,amount,order_sequence,monthly_sales_rank,running_total,moving_avg_4_orders,previous_order,next_order,first_order_amount
0,O1001,C001,John Doe,2024-01-15,120.5,1,1,120.5,120.500000,0.0,75.0,120.5
1,O1002,C001,John Doe,2024-02-20,75.0,2,1,195.5,97.750000,120.5,50.0,120.5
2,O1003,C001,John Doe,2024-03-05,50.0,3,1,245.5,81.833333,75.0,0.0,120.5
3,O1004,C002,Jane Smith,2024-04-25,200.0,1,1,200.0,200.000000,0.0,150.0,200.0
4,O1005,C002,Jane Smith,2024-05-01,150.0,2,1,350.0,175.000000,200.0,0.0,200.0
